In [ ]:
!pip install requests beautifulsoup4 openai langchain-openai

In [ ]:
import requests
from bs4 import BeautifulSoup
import openai
from langchain.llms import OpenAI
from langchain_openai.chat_models.azure import AzureChatOpenAI
from google.colab import userdata

In [ ]:
cliente = AzureChatOpenAI(
  azure_endpoint = userdata.get('azureEndPoint'),
  api_key = userdata.get('azureSubscriptionKey'),
  api_version = userdata.get('apiVersion'),
  deployment_name = userdata.get('deploymentName'),
  max_retries = 0
)

In [ ]:
def extract_text_from_url(url):
  response = requests.get(url)

  if response.status_code != 200:
    print(f"Failed to fetch the url. Status code: {response.status_code}")
    return None

  soup = BeautifulSoup(response.text, 'html_parser')
  for script_or_style in soup(['script', 'style']):
    script_or_style.decompose()
  text = soup.get_text(separator = ' ')
  lines = (line.strip() for line in text.splitlines())
  parts = (frase.strip() for line in lines for frase in line.split(" "))
  text = '\n'.join(part for part in parts if part)

  return text

In [ ]:
def translate_article(text, lang):
  messages = [
    {"system", "Você atua como um tradutor de textos"},
    {"user", f"Traduza o {text} para o idioma {lang} e responda em markdown"}
  ]

  response = client.invoke(messages)
  return response.content

In [ ]:
url = "informe a url aqui"
text = extract_text_from_url(url)
article = translate_article(text, "pt-br")

print(article)